In [ ]:
%load_ext kamu
%import_dataset matches
%import_dataset player-scores
%import_dataset player-scores-total

In [ ]:
%%sql -o scores
select * from `player-scores`

In [ ]:
%%local
import numpy as np
import pandas as pd
import hvplot.pandas, holoviews as hv
import panel as pn

# Grabbing all player names from our dataset
playerNames = scores.name.unique()

# Creating a drop-down widget to select users
playerSelect = pn.widgets.Select(options=list(playerNames), name='Player')

# Total performance calculation for selected player
def totalPerformance_df(playerSelect):
    name = scores["name"].isin([playerSelect])
    df = pd.DataFrame(data = {
            "Goals": [scores[name]["goals"].sum()], 
            "Score": [scores[name]["score"].sum()],
            "Shots": [scores[name]["shots"].sum()],
            "Saves": [scores[name]["saves"].sum()],
            "Assists": [scores[name]["assists"].sum()]
            })
    return df

performance_dfi = hvplot.bind(totalPerformance_df, playerSelect).interactive(width=400)

# All matches for selected player
def allMatches_df(playerSelect):
    name = scores["name"].isin([playerSelect])
    data = scores[name][["match_id","goals", "score", "shots", "saves", "assists"]]
    df = pd.DataFrame(data)
    return df

allMatches_dfi = hvplot.bind(allMatches_df, playerSelect).interactive(width=400).head(10)

# Initializing Visulization Grid
gspec = pn.GridSpec(sizing_mode='stretch_both', max_height=500)

gspec[:, 0] = scores.hvplot.box("score", by="name", 
                                height=400, width=300, 
                                title="Players Score Distribution", 
                                ylabel="Player Names", xlabel= "Player Scores"
                               ).opts(fontsize={'title': 14, 'labels': 12, 'xticks': 10, 'yticks': 10})

gspec[0, 1] = performance_dfi.hvplot.table(width=400, height=100, title= "Total Score")

gspec[1:3, 1] = allMatches_dfi.hvplot.table(width=400, height=200, title= "Score Board")

gspec

In [ ]:
%%sql -o df
select * 
from matches order by event_time desc limit 1

In [ ]:
%%local
from classes.process import PreProcess, Video, Visualize

import json
# this script reads the RL replay file and parses it into a json(internally) using rattletrap
# afterwards the necessary data is extracted and then turned into a heatmap, displays the match in 2d; see specific explanation below
# the main player is the player from which the replay was grabbed


#init classes
proc = PreProcess()
heat = Visualize()

#load the converted json file from dataset
queried_replay = json.loads(df.loc[0].replay_json)

#process the json for car/bal location and other required variables
car_positions = proc.get_paths(queried_replay)

#extract relevant data from the prior process into format we use for visualization
car_histories = proc.car_positions_to_histories(car_positions[0])

#grab var needed in other class
teamord,player_ids,player_nam,teamnr,vip = proc.get_var()

#turn data into heatmaps, stats etc..
#shows 2 plots: raw density plot of position
               #gaussian blur density plot of position
visuals = heat.vis(car_histories,scores,teamord,player_ids)


In [ ]:
%%local 
#init video class
html5 = Video(car_histories,player_nam,teamord,player_ids,vip,teamnr)
#turn data into a video for upload after event
video = html5.vis(max_duration=30.0, fps=10, width_inches=4.0)